# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [331]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:
# Output File (CSV)
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [36]:
# create a params dict that will be updated with new city each iteration
params = {"key": g_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in cities_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

In [37]:
gmaps.configure(api_key=g_key)

In [92]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
cities_df = cities_df.dropna()
Humidity = cities_df["Humidity"]

In [93]:
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
narrowed_df = cities_df.loc[cities_df['Lat'] < 80 ]
marrowed_df = narrowed_df.loc[narrowed_df['Lat'] > 70]
warrowed_df = marrowed_df.loc[marrowed_df['Wind Speed'] < 10]
weather_df =  marrowed_df.loc[marrowed_df['Cloudiness'] == 0]
weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
13,13,saskylakh,0,RU,1558378757,91,71.97,114.09,4.92,2.26
179,179,chokurdakh,0,RU,1558378788,89,70.62,147.90,7.08,5.99
309,309,upernavik,0,GL,1558378846,85,72.79,-56.15,29.22,11.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [211]:
# set up additional columns to hold information
hotels_df = cities_df['Hotel Name'] = ""
hotels_df = cities_df
# hotels_df

In [ ]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


target_radius = 5000
params = {
    "location": "39.952583,-75.16522",  # philadelphia coords
    #"rankby": "distance",
    "type": "lodging",
    "key": g_key,
    "radius": target_radius
}
# use iterrows to iterate through pandas dataframe
for index, row in cities_df.iterrows():

    # get restaurant type from df
    vacat_type = row['City']
    params["location"] = f"{row['Lat']},{row['Lng']}" 

    # add keyword to params dict

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {vacat_type}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"The closest hotel near {vacat_type} is {results[0]['name']}.")
        hotels_df.loc[index, 'Hotel Name'] = results[0]['name']
 
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [247]:
narrowed_df = hotels_df.loc[cities_df['Lat'] < 80 ]
marrowed_df = narrowed_df.loc[narrowed_df['Lat'] > 65]
warrowed_df = marrowed_df.loc[marrowed_df['Wind Speed'] < 10]
hotel_df =  marrowed_df.loc[marrowed_df['Cloudiness'] == 0]
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
13,13,saskylakh,0,RU,1558378757,91,71.97,114.09,4.92,2.26,
179,179,chokurdakh,0,RU,1558378788,89,70.62,147.90,7.08,5.99,
191,191,naryan-mar,0,RU,1558378791,90,67.67,53.09,27.24,8.19,Hostel Agat
219,219,beisfjord,0,NO,1558378796,51,68.38,17.60,60.80,11.41,
220,220,ust-tsilma,0,RU,1558378796,58,65.44,52.15,35.88,6.11,U Praskov'i
251,251,leningradskiy,0,RU,1558378827,80,69.38,178.42,22.20,6.38,
309,309,upernavik,0,GL,1558378846,85,72.79,-56.15,29.22,11.59,Gina's Guesthouse - Upernavik
359,359,sorland,0,NO,1558378858,71,67.67,12.69,53.60,5.82,Sjybrygga fiskecamp


In [240]:
len(cities_df)

544

Retrieving Results for Index 0: ostrovnoy.
Missing field/result... skipping.
------------
Retrieving Results for Index 1: mahebourg.
The closest hotel near mahebourg is Shandrani Beachcomber Resort & Spa.
------------
Retrieving Results for Index 2: qaanaaq.
The closest hotel near qaanaaq is Qaanaaq Hotel.
------------
Retrieving Results for Index 3: zhuhai.
Missing field/result... skipping.
------------
Retrieving Results for Index 4: cape town.
The closest hotel near cape town is Southern Sun Cape Sun.
------------
Retrieving Results for Index 5: atuona.
The closest hotel near atuona is Villa Enata.
------------
Retrieving Results for Index 6: flinders.
The closest hotel near flinders is Shellharbour Resort & Conference Centre.
------------
Retrieving Results for Index 8: tiksi.
The closest hotel near tiksi is Umka.
------------
Retrieving Results for Index 9: pushkinskiye gory.
The closest hotel near pushkinskiye gory is The recreation center "Pushkinogorye".
------------
Retrieving

The closest hotel near pontes e lacerda is Hotel Bressan.
------------
Retrieving Results for Index 77: khor.
The closest hotel near khor is Kafe, Gostinitsa Inzhir.
------------
Retrieving Results for Index 78: crotone.
The closest hotel near crotone is Helios Hotel.
------------
Retrieving Results for Index 79: baglung.
The closest hotel near baglung is Hotel Peace Palace, Baglung, Nepal.
------------
Retrieving Results for Index 80: raglan.
The closest hotel near raglan is Raglan Sunset Motel.
------------
Retrieving Results for Index 81: jamestown.
The closest hotel near jamestown is Jamestown Motel Terraces.
------------
Retrieving Results for Index 82: sur.
The closest hotel near sur is Sur Plaza Hotel.
------------
Retrieving Results for Index 83: araouane.
Missing field/result... skipping.
------------
Retrieving Results for Index 84: emerald.
The closest hotel near emerald is The Irish Village.
------------
Retrieving Results for Index 85: launceston.
The closest hotel near la

The closest hotel near sinnamary is ALETHEIA Appartement Sinnamary.
------------
Retrieving Results for Index 153: porto novo.
The closest hotel near porto novo is Harmonia.
------------
Retrieving Results for Index 154: havoysund.
The closest hotel near havoysund is Arctic View Glamping & Restaurant.
------------
Retrieving Results for Index 155: bjornevatn.
The closest hotel near bjornevatn is Snowhotel Kirkenes & Gamme Northern Lights Cabins.
------------
Retrieving Results for Index 156: narsaq.
The closest hotel near narsaq is Hotel Narsaq.
------------
Retrieving Results for Index 157: charlestown.
The closest hotel near charlestown is Charlestown Breachway.
------------
Retrieving Results for Index 158: trat.
The closest hotel near trat is Hotel Toscana Trat.
------------
Retrieving Results for Index 159: manado.
The closest hotel near manado is Aryaduta Hotel.
------------
Retrieving Results for Index 160: trapani.
The closest hotel near trapani is Hotel Tiziano.
------------
R

The closest hotel near ekimchan is Mini-Gostinitsa "Ekimchan".
------------
Retrieving Results for Index 225: carnarvon.
The closest hotel near carnarvon is Maritza Bed & Breakfast.
------------
Retrieving Results for Index 226: saint-denis.
The closest hotel near saint-denis is Eurohotel Paris Nord.
------------
Retrieving Results for Index 227: tabas.
The closest hotel near tabas is شرکت معماری مهراز شایگان آراد.
------------
Retrieving Results for Index 228: san pedro.
The closest hotel near san pedro is Hotel San Pedro Palace.
------------
Retrieving Results for Index 229: aketi.
Missing field/result... skipping.
------------
Retrieving Results for Index 230: moses lake.
The closest hotel near moses lake is Ramada by Wyndham Moses Lake.
------------
Retrieving Results for Index 231: mocuba.
The closest hotel near mocuba is Phama Lodge.
------------
Retrieving Results for Index 232: hamilton.
The closest hotel near hamilton is Sheraton Hamilton Hotel.
------------
Retrieving Results

The closest hotel near mikkeli is Original Sokos Hotel Vaakuna, Mikkeli.
------------
Retrieving Results for Index 299: jiuquan.
The closest hotel near jiuquan is Jiuquan Hotel.
------------
Retrieving Results for Index 300: betafo.
The closest hotel near betafo is FJKM ATSIMONIAVOKO MIFANKATIA.
------------
Retrieving Results for Index 301: aitape.
The closest hotel near aitape is 郭超本度假酒店.
------------
Retrieving Results for Index 302: alofi.
The closest hotel near alofi is Taloa Heights.
------------
Retrieving Results for Index 303: yulara.
The closest hotel near yulara is Desert Gardens Hotel - Ayers Rock Resort.
------------
Retrieving Results for Index 304: kvarkeno.
The closest hotel near kvarkeno is Ooo "Kvarkeno".
------------
Retrieving Results for Index 305: salinopolis.
The closest hotel near salinopolis is Salinas Park Resort Maçarico AP101.
------------
Retrieving Results for Index 306: chunhuhub.
The closest hotel near chunhuhub is Hotelito Noe.
------------
Retrieving R

The closest hotel near carballo is Pazo do Souto.
------------
Retrieving Results for Index 372: shimoda.
The closest hotel near shimoda is Shimoda Tokyu Hotel.
------------
Retrieving Results for Index 373: novonukutskiy.
The closest hotel near novonukutskiy is Gostinitsa.
------------
Retrieving Results for Index 374: reguengos de monsaraz.
The closest hotel near reguengos de monsaraz is Residencial O Gato.
------------
Retrieving Results for Index 375: ocean city.
The closest hotel near ocean city is Fairfield Inn & Suites Fort Walton Beach - Eglin AFB Hotel.
------------
Retrieving Results for Index 376: qinhuangdao.
The closest hotel near qinhuangdao is Qinhuangdao Peninsula Seasons Hotel And Apartment.
------------
Retrieving Results for Index 377: muskegon.
The closest hotel near muskegon is Shoreline Inn & Conference Center, An Ascend Hotel Collection Mem.
------------
Retrieving Results for Index 378: kargasok.
The closest hotel near kargasok is Hotel Kargasok.
------------
Re

The closest hotel near birjand is تاکسی سرویس آرامش.
------------
Retrieving Results for Index 445: ingham.
The closest hotel near ingham is Lees Hotel - the Pub with No Beer.
------------
Retrieving Results for Index 447: tilichiki.
Missing field/result... skipping.
------------
Retrieving Results for Index 448: doha.
The closest hotel near doha is Doha Marriott Hotel.
------------
Retrieving Results for Index 449: makakilo city.
The closest hotel near makakilo city is Marriott's Ko Olina Beach Club.
------------
Retrieving Results for Index 450: lorengau.
The closest hotel near lorengau is Lorengau Harbourside Hotel.
------------
Retrieving Results for Index 451: anan.
The closest hotel near anan is Royal Garden Hotel.
------------
Retrieving Results for Index 452: synya.
Missing field/result... skipping.
------------
Retrieving Results for Index 453: thanatpin.
The closest hotel near thanatpin is Shwe War Htun.
------------
Retrieving Results for Index 454: altagracia de orituco.
Th

The closest hotel near bonito is Marruá Hotel.
------------
Retrieving Results for Index 519: saint anthony.
The closest hotel near saint anthony is Three Bears Inn.
------------
Retrieving Results for Index 520: high prairie.
The closest hotel near high prairie is Royal Star Motor Inn.
------------
Retrieving Results for Index 521: high rock.
The closest hotel near high rock is Hideaway Bahamas Beach Club.
------------
Retrieving Results for Index 522: raudeberg.
The closest hotel near raudeberg is Nordre Silda..
------------
Retrieving Results for Index 523: kununurra.
The closest hotel near kununurra is Hotel Kununurra.
------------
Retrieving Results for Index 524: mogadishu.
The closest hotel near mogadishu is Olympic Hotel.
------------
Retrieving Results for Index 525: mecca.
The closest hotel near mecca is Makkah Clock Royal Tower, A Fairmont Hotel.
------------
Retrieving Results for Index 526: viedma.
The closest hotel near viedma is Hotel Cristal.
------------
Retrieving Res

In [242]:
 response = requests.get(base_url, params=params).json()

In [248]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
13,13,saskylakh,0,RU,1558378757,91,71.97,114.09,4.92,2.26,
179,179,chokurdakh,0,RU,1558378788,89,70.62,147.90,7.08,5.99,
191,191,naryan-mar,0,RU,1558378791,90,67.67,53.09,27.24,8.19,Hostel Agat
219,219,beisfjord,0,NO,1558378796,51,68.38,17.60,60.80,11.41,
220,220,ust-tsilma,0,RU,1558378796,58,65.44,52.15,35.88,6.11,U Praskov'i
251,251,leningradskiy,0,RU,1558378827,80,69.38,178.42,22.20,6.38,
309,309,upernavik,0,GL,1558378846,85,72.79,-56.15,29.22,11.59,Gina's Guesthouse - Upernavik
359,359,sorland,0,NO,1558378858,71,67.67,12.69,53.60,5.82,Sjybrygga fiskecamp


In [237]:
# # find the closest restaurant of each type to coordinates

# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# target_radius = 5000
# params = {
#     "location": "39.952583,-75.16522",  # philadelphia coords
#     #"rankby": "distance",
#     "type": "lodging",
#     "key": g_key,
#     "radius": target_radius
# }
# # use iterrows to iterate through pandas dataframe
# for index, row in cities_df.iterrows():

#     # get restaurant type from df
#     vacat_type = row['City']
#     params["location"] = f"{row['Lat']},{row['Lng']}" 

#     # add keyword to params dict

#     # assemble url and make API request
#     print(f"Retrieving Results for Index {index}: {vacat_type}.")
#     response = requests.get(base_url, params=params).json()
    
#     # extract results
#     results = response['results']
    
#     try:
#         print(f"The closest hotel near {vacat_type} is {results[0]['name']}.")
#         hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
 
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
        
#     print("------------")

Retrieving Results for Index 0: ostrovnoy.
Missing field/result... skipping.
------------
Retrieving Results for Index 1: mahebourg.
The closest hotel near mahebourg is Shandrani Beachcomber Resort & Spa.
------------
Retrieving Results for Index 2: qaanaaq.
The closest hotel near qaanaaq is Qaanaaq Hotel.

<ipython-input-237-3624701b9a6a>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[index, 'Hotel Name'] = results[0]['name']



------------
Retrieving Results for Index 3: zhuhai.
Missing field/result... skipping.
------------
Retrieving Results for Index 4: cape town.
The closest hotel near cape town is Southern Sun Cape Sun.
------------
Retrieving Results for Index 5: atuona.
The closest hotel near atuona is Villa Enata.
------------
Retrieving Results for Index 6: flinders.
The closest hotel near flinders is Shellharbour Resort & Conference Centre.
------------
Retrieving Results for Index 8: tiksi.
The closest hotel near tiksi is Umka.
------------
Retrieving Results for Index 9: pushkinskiye gory.
The closest hotel near pushkinskiye gory is The recreation center "Pushkinogorye".
------------
Retrieving Results for Index 10: aykhal.
The closest hotel near aykhal is Hotel Astoria.
------------
Retrieving Results for Index 11: castro.
The closest hotel near castro is Hotel Esmeralda.
------------
Retrieving Results for Index 12: avarua.
The closest hotel near avarua is Paradise Inn.
------------
Retrieving

The closest hotel near raglan is Raglan Sunset Motel.
------------
Retrieving Results for Index 81: jamestown.
The closest hotel near jamestown is Jamestown Motel Terraces.
------------
Retrieving Results for Index 82: sur.
The closest hotel near sur is Sur Plaza Hotel.
------------
Retrieving Results for Index 83: araouane.
Missing field/result... skipping.
------------
Retrieving Results for Index 84: emerald.
The closest hotel near emerald is The Irish Village.
------------
Retrieving Results for Index 85: launceston.
The closest hotel near launceston is Hotel Grand Chancellor Launceston.
------------
Retrieving Results for Index 86: nikolskoye.
The closest hotel near nikolskoye is Tourist House - Sablino.
------------
Retrieving Results for Index 87: ambilobe.
The closest hotel near ambilobe is Hôtel & Restaurant DIANA Ambilobe.
------------
Retrieving Results for Index 88: cidreira.
The closest hotel near cidreira is Hotel Castelo.
------------
Retrieving Results for Index 89: ter

The closest hotel near bjornevatn is Snowhotel Kirkenes & Gamme Northern Lights Cabins.
------------
Retrieving Results for Index 156: narsaq.
The closest hotel near narsaq is Hotel Narsaq.
------------
Retrieving Results for Index 157: charlestown.
The closest hotel near charlestown is Charlestown Breachway.
------------
Retrieving Results for Index 158: trat.
The closest hotel near trat is Hotel Toscana Trat.
------------
Retrieving Results for Index 159: manado.
The closest hotel near manado is Aryaduta Hotel.
------------
Retrieving Results for Index 160: trapani.
The closest hotel near trapani is Hotel Tiziano.
------------
Retrieving Results for Index 161: sao joao da barra.
The closest hotel near sao joao da barra is Pousada Porto De Canoas.
------------
Retrieving Results for Index 162: saint-philippe.
The closest hotel near saint-philippe is Camping St-Andre.
------------
Retrieving Results for Index 163: haines junction.
The closest hotel near haines junction is Parkside Inn.

------------
Retrieving Results for Index 228: san pedro.
The closest hotel near san pedro is Hotel San Pedro Palace.
------------
Retrieving Results for Index 229: aketi.
Missing field/result... skipping.
------------
Retrieving Results for Index 230: moses lake.
The closest hotel near moses lake is Ramada by Wyndham Moses Lake.
------------
Retrieving Results for Index 231: mocuba.
The closest hotel near mocuba is Phama Lodge.
------------
Retrieving Results for Index 232: hamilton.
The closest hotel near hamilton is Sheraton Hamilton Hotel.
------------
Retrieving Results for Index 233: san juan del sur.
The closest hotel near san juan del sur is Nuestra Casa.
------------
Retrieving Results for Index 234: katobu.
The closest hotel near katobu is La Ode Husein.
------------
Retrieving Results for Index 235: banjar.
The closest hotel near banjar is OYO 2488 Villa Tirta Melati Syariah.
------------
Retrieving Results for Index 236: provideniya.
Missing field/result... skipping.
------

The closest hotel near alofi is Taloa Heights.
------------
Retrieving Results for Index 303: yulara.
The closest hotel near yulara is Desert Gardens Hotel - Ayers Rock Resort.
------------
Retrieving Results for Index 304: kvarkeno.
The closest hotel near kvarkeno is Ooo "Kvarkeno".
------------
Retrieving Results for Index 305: salinopolis.
The closest hotel near salinopolis is Salinas Park Resort Maçarico AP101.
------------
Retrieving Results for Index 306: chunhuhub.
The closest hotel near chunhuhub is Hotelito Noe.
------------
Retrieving Results for Index 307: dunedin.
The closest hotel near dunedin is Scenic Hotel Southern Cross.
------------
Retrieving Results for Index 308: nioro.
Missing field/result... skipping.
------------
Retrieving Results for Index 309: upernavik.
The closest hotel near upernavik is Gina's Guesthouse - Upernavik.
------------
Retrieving Results for Index 310: airai.
The closest hotel near airai is Juvinal Martin.
------------
Retrieving Results for Ind

The closest hotel near ocean city is Fairfield Inn & Suites Fort Walton Beach - Eglin AFB Hotel.
------------
Retrieving Results for Index 376: qinhuangdao.
The closest hotel near qinhuangdao is Qinhuangdao Peninsula Seasons Hotel And Apartment.
------------
Retrieving Results for Index 377: muskegon.
The closest hotel near muskegon is Shoreline Inn & Conference Center, An Ascend Hotel Collection Mem.
------------
Retrieving Results for Index 378: kargasok.
The closest hotel near kargasok is Hotel Kargasok.
------------
Retrieving Results for Index 379: auki.
Missing field/result... skipping.
------------
Retrieving Results for Index 380: kayseri.
The closest hotel near kayseri is Bent Hotel.
------------
Retrieving Results for Index 381: quanzhou.
The closest hotel near quanzhou is Super 8 Hotel Quanzhou new station shop.
------------
Retrieving Results for Index 382: nago.
The closest hotel near nago is Hotel Garda.
------------
Retrieving Results for Index 383: medicine hat.
The clo

The closest hotel near doha is Doha Marriott Hotel.
------------
Retrieving Results for Index 449: makakilo city.
The closest hotel near makakilo city is Marriott's Ko Olina Beach Club.
------------
Retrieving Results for Index 450: lorengau.
The closest hotel near lorengau is Lorengau Harbourside Hotel.
------------
Retrieving Results for Index 451: anan.
The closest hotel near anan is Royal Garden Hotel.
------------
Retrieving Results for Index 452: synya.
Missing field/result... skipping.
------------
Retrieving Results for Index 453: thanatpin.
The closest hotel near thanatpin is Shwe War Htun.
------------
Retrieving Results for Index 454: altagracia de orituco.
The closest hotel near altagracia de orituco is Posada Quinta Mi Refugio.
------------
Retrieving Results for Index 455: anqing.
The closest hotel near anqing is Hongtian Hotel.
------------
Retrieving Results for Index 456: nome.
The closest hotel near nome is Silver Spur Mobile Home & RV Park.
------------
Retrieving Re

The closest hotel near high rock is Hideaway Bahamas Beach Club.
------------
Retrieving Results for Index 522: raudeberg.
The closest hotel near raudeberg is Nordre Silda..
------------
Retrieving Results for Index 523: kununurra.
The closest hotel near kununurra is Hotel Kununurra.
------------
Retrieving Results for Index 524: mogadishu.
The closest hotel near mogadishu is Olympic Hotel.
------------
Retrieving Results for Index 525: mecca.
The closest hotel near mecca is Makkah Clock Royal Tower, A Fairmont Hotel.
------------
Retrieving Results for Index 526: viedma.
The closest hotel near viedma is Hotel Cristal.
------------
Retrieving Results for Index 527: tomatlan.
The closest hotel near tomatlan is Norma franco.
------------
Retrieving Results for Index 528: lyuban.
Missing field/result... skipping.
------------
Retrieving Results for Index 529: bonavista.
The closest hotel near bonavista is Abbott's Bed & Breakfast.
------------
Retrieving Results for Index 530: hirara.
The

In [335]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,"(Lat, Lng)"
13,13,saskylakh,0,RU,1558378757,91,71.97,114.09,4.92,2.26,,Hotel Name
179,179,chokurdakh,0,RU,1558378788,89,70.62,147.90,7.08,5.99,,Hotel Name
191,191,naryan-mar,0,RU,1558378791,90,67.67,53.09,27.24,8.19,Hostel Agat,Hotel Name
219,219,beisfjord,0,NO,1558378796,51,68.38,17.60,60.80,11.41,,Hotel Name
220,220,ust-tsilma,0,RU,1558378796,58,65.44,52.15,35.88,6.11,U Praskov'i,Hotel Name
251,251,leningradskiy,0,RU,1558378827,80,69.38,178.42,22.20,6.38,,Hotel Name
309,309,upernavik,0,GL,1558378846,85,72.79,-56.15,29.22,11.59,Gina's Guesthouse - Upernavik,Hotel Name
359,359,sorland,0,NO,1558378858,71,67.67,12.69,53.60,5.82,Sjybrygga fiskecamp,Hotel Name


In [250]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [251]:
locations

,Lat,Lng
13,71.97,114.09
179,70.62,147.90
191,67.67,53.09
219,68.38,17.60
220,65.44,52.15
251,69.38,178.42
309,72.79,-56.15
359,67.67,12.69


In [252]:
# Add marker layer ontop of heat map


# Display figure


In [326]:
plant_locations = [plant['location'] for plant in nuclear_power_plants]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Number reactors</dt><dd>{active_reactors}</dd>
</dl>
"""
plant_info = [info_box_template.format(**plant) for plant in nuclear_power_plants]
info_box_template.format(**nuclear_power_plants[4])

'\n<dl>\n<dt>Name</dt><dd>Doel</dd>\n<dt>Number reactors</dt><dd>4</dd>\n</dl>\n'

In [344]:
# gmaps.configure()

# nuclear_power_plants = [
#     {'name': 'Atucha', 'location': (-34.0, -59.167), 'active_reactors': 1},
#     {'name': 'Embalse', 'location': (-32.2333, -64.4333), 'active_reactors': 1},
#     {'name': 'Armenia', 'location': (40.167, 44.133), 'active_reactors': 1},
#     {'name': 'Br', 'location': (51.217, 5.083), 'active_reactors': 1},
#     {'name': 'Doel', 'location': (51.333, 4.25), 'active_reactors': 4},
#     {'name': 'Tihange', 'location': (50.517, 5.283), 'active_reactors': 3}
# ]

# plant_locations = [plant['location'] for plant in nuclear_power_plants]
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{name}</dd>
# <dt>Number reactors</dt><dd>{active_reactors}</dd>
# </dl>
# """
# plant_info = [info_box_template.format(**plant) for plant in nuclear_power_plants]

# marker_layer = gmaps.marker_layer(plant_locations, info_box_content=plant_info)
# fig = gmaps.figure()
# fig.add_layer(marker_layer)
# fig


In [345]:
gmaps.configure()

hotel_info = json.loads(hotel_df.to_json(orient='records'))

hotel_locations = [(hotel["Lat"],hotel["Lng"]) for hotel in hotel_info]
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
city_info = [info_box_template.format(**hotel) for hotel in hotel_info]
heat_layer = gmaps.heatmap_layer(hotel_df[["Lat", "Lng"]],  
                                 dissipating=False, max_intensity=1,
                                 point_radius = 5)
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=city_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [333]:
json.loads(hotel_df.to_json(orient='records'))

[{'City_ID': 13,
  'City': 'saskylakh',
  'Cloudiness': 0,
  'Country': 'RU',
  'Date': 1558378757,
  'Humidity': 91,
  'Lat': 71.97,
  'Lng': 114.09,
  'Max Temp': 4.92,
  'Wind Speed': 2.26,
  'Hotel Name': '',
  "('Lat', 'Lng')": 'Hotel Name'},
 {'City_ID': 179,
  'City': 'chokurdakh',
  'Cloudiness': 0,
  'Country': 'RU',
  'Date': 1558378788,
  'Humidity': 89,
  'Lat': 70.62,
  'Lng': 147.9,
  'Max Temp': 7.08,
  'Wind Speed': 5.99,
  'Hotel Name': '',
  "('Lat', 'Lng')": 'Hotel Name'},
 {'City_ID': 191,
  'City': 'naryan-mar',
  'Cloudiness': 0,
  'Country': 'RU',
  'Date': 1558378791,
  'Humidity': 90,
  'Lat': 67.67,
  'Lng': 53.09,
  'Max Temp': 27.24,
  'Wind Speed': 8.19,
  'Hotel Name': 'Hostel Agat',
  "('Lat', 'Lng')": 'Hotel Name'},
 {'City_ID': 219,
  'City': 'beisfjord',
  'Cloudiness': 0,
  'Country': 'NO',
  'Date': 1558378796,
  'Humidity': 51,
  'Lat': 68.38,
  'Lng': 17.6,
  'Max Temp': 60.8,
  'Wind Speed': 11.41,
  'Hotel Name': '',
  "('Lat', 'Lng')": 'Hotel Na

In [334]:
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,"(Lat, Lng)"
13,13,saskylakh,0,RU,1558378757,91,71.97,114.09,4.92,2.26,,Hotel Name
179,179,chokurdakh,0,RU,1558378788,89,70.62,147.90,7.08,5.99,,Hotel Name
191,191,naryan-mar,0,RU,1558378791,90,67.67,53.09,27.24,8.19,Hostel Agat,Hotel Name
219,219,beisfjord,0,NO,1558378796,51,68.38,17.60,60.80,11.41,,Hotel Name
220,220,ust-tsilma,0,RU,1558378796,58,65.44,52.15,35.88,6.11,U Praskov'i,Hotel Name
